In [1]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
import requests
import json
import os
import re
import time
from tqdm import tqdm
from glob import glob
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
def download(url, file_name, retry = 3):
    if os.path.exists(file_name) and os.path.getsize(file_name) > 50000:
        return
    with open(file_name, "wb") as file:
        for k in range(retry):
            try:
                response = requests.get(url, verify = False, timeout = 360)
                file.write(response.content)
                return 
            except Exception as e:
                print(k, e)
                time.sleep(0.5)

In [4]:
files = glob('pages/*.json')
len(files)

18787

In [5]:
pdfs = []
for f in files:
    with open(f) as fopen:
        pdfs.extend(json.load(fopen))
        
pdfs = list(set(pdfs))

In [6]:
filtered_pdfs = []
for p in tqdm(pdfs):
    if len(re.findall( r'[0-9]+(?:\.[0-9]+){3}', p)):
        continue
    if not p.endswith('.pdf'):
        continue
    if 'ade.sagepub.com' in p:
        continue
    filtered_pdfs.append(p)
    
filtered_pdfs = sorted(list(set(filtered_pdfs)))
len(filtered_pdfs)

100%|███████████████████████████████| 235433/235433 [00:00<00:00, 677665.27it/s]


235130

In [7]:
ukm = [f for f in filtered_pdfs if 'ukm.my' in f]
len(ukm)

18101

In [8]:
# !rm -rf pdf
!mkdir ukm

mkdir: cannot create directory ‘ukm’: File exists


In [9]:
max_worker = 1

ukm = sorted(list(ukm))
for i in tqdm(range(0, len(ukm), max_worker)):
    urls = [(url, os.path.join('ukm', f'{i}-{no}.pdf')) for no, url in enumerate(ukm[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(download, url[0], url[1]): url for url in urls}

        for future in as_completed(futures):
            future.result()

 99%|████████████████████████████████████▋| 17933/18101 [01:40<00:46,  3.59it/s]

0 HTTPConnectionPool(host='www.fsk.ukm.my', port=80): Max retries exceeded with url: /jurnal/jilid%206%282%292008/Chap%203.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fe9b82fed30>: Failed to establish a new connection: [Errno -2] Name or service not known'))
1 HTTPConnectionPool(host='www.fsk.ukm.my', port=80): Max retries exceeded with url: /jurnal/jilid%206%282%292008/Chap%203.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fe9b82feb20>: Failed to establish a new connection: [Errno -2] Name or service not known'))
2 HTTPConnectionPool(host='www.fsk.ukm.my', port=80): Max retries exceeded with url: /jurnal/jilid%206%282%292008/Chap%203.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fe9b82fef70>: Failed to establish a new connection: [Errno -2] Name or service not known'))
0 HTTPConnectionPool(host='www.fsk.ukm.my', port=80): Max retries exceeded with url: /jurnal/jilid%206%282

 99%|████████████████████████████████████▋| 17933/18101 [01:59<00:46,  3.59it/s]

1 HTTPConnectionPool(host='www.fsk.ukm.my', port=80): Max retries exceeded with url: /jurnal/jilid%207%281%292009/Chapter2.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fe9b82fedc0>: Failed to establish a new connection: [Errno -2] Name or service not known'))
2 HTTPConnectionPool(host='www.fsk.ukm.my', port=80): Max retries exceeded with url: /jurnal/jilid%207%281%292009/Chapter2.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fe9b82feb20>: Failed to establish a new connection: [Errno -2] Name or service not known'))


 99%|████████████████████████████████████▋| 17947/18101 [02:00<01:15,  2.05it/s]

...

100%|█████████████████████████████████▊| 18027/18101 [28:17<2:05:30, 101.76s/it]

0 ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
1 ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
2 ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


100%|████████████████████████████████████▉| 18057/18101 [31:39<04:47,  6.54s/it]

0 ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
1 ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
2 ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


100%|█████████████████████████████████████| 18101/18101 [35:02<00:00,  8.61it/s]
